<a href="https://colab.research.google.com/github/phatdo05/AI220/blob/main/btl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab.patches import cv2_imshow
import time

In [ ]:

# Install necessary libraries
!pip install opencv-python
!pip install matplotlib

import cv2
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from google.colab import files
import matplotlib.pyplot as plt

## 1. Load trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/Colab Notebooks/best (5).pt')  # Load pretrained model

## 2. Define behavior classes (all considered as negative/warning behaviors)
class_names = {
        0: "Looking_front",
        1: "Looking_up",
        2: "Mobile",
        3: "Passing_cheat",
        4: "backword",
        5: "c",
        6: "five",
        7: "four",
        8: "head",
        9: "head_down",
        10: "looking_left",
        11: "looking_right",
        12: "three",
        13: "two"
}

# All behaviors are considered negative/warning
negative_behaviors = list(class_names.values())

## 3. Behavior analysis function
def analyze_behavior(detections):
    behavior_counts = {name: 0 for name in class_names.values()}
    warning_count = 0

    for detection in detections:
        class_id = detection.boxes.cls.cpu().numpy()[0]
        class_name = class_names[int(class_id)]
        behavior_counts[class_name] += 1
        warning_count += 1  # All detections count as warnings

    return behavior_counts, warning_count

## 4. Simultaneous behavior detection function
def detect_simultaneous_behaviors(detections, iou_threshold=0.5):
    simultaneous = []
    boxes = [detection.boxes.xyxy.cpu().numpy()[0] for detection in detections]
    classes = [detection.boxes.cls.cpu().numpy()[0] for detection in detections]

    for i in range(len(detections)):
        for j in range(i+1, len(detections)):
            box1 = boxes[i]
            box2 = boxes[j]

            # Calculate IoU
            x1 = max(box1[0], box2[0])
            y1 = max(box1[1], box2[1])
            x2 = min(box1[2], box2[2])
            y2 = min(box1[3], box2[3])

            intersection = max(0, x2 - x1) * max(0, y2 - y1)
            area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
            area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
            union = area1 + area2 - intersection
            iou = intersection / union if union > 0 else 0

            if iou > iou_threshold:
                class1 = class_names[int(classes[i])]
                class2 = class_names[int(classes[j])]
                simultaneous.append((class1, class2))

    return simultaneous

## 5. Video processing function
def process_video(video_path, output_path='output.mp4'):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Create VideoWriter for output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_warnings = 0
    behavior_history = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect objects with YOLOv8
        results = model(frame)

        # Analyze behavior
        detections = results[0]
        behavior_counts, warnings = analyze_behavior(detections)
        simultaneous = detect_simultaneous_behaviors(detections)

        # Update statistics
        total_warnings += warnings
        behavior_history.append(behavior_counts)

        # Draw bounding boxes and info
        annotated_frame = results[0].plot()

        # Display behavior info (all warnings)
        info_text = f"Warning Behaviors: {warnings} | Frame: {frame_count}"
        cv2.putText(annotated_frame, info_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)  # Red color for warnings

        # Display simultaneous behaviors
        for i, (beh1, beh2) in enumerate(simultaneous):
            cv2.putText(annotated_frame, f"WARNING: {beh1} + {beh2}", (10, 60 + i*30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Write frame to output video
        out.write(annotated_frame)
        frame_count += 1

        # Display progress
        if frame_count % 50 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    out.release()

    # Generate report
    generate_report(behavior_history, total_warnings, frame_count)

    return output_path

## 6. Report generation function
def generate_report(behavior_history, total_warnings, total_frames):
    # Calculate total occurrences of each behavior
    total_behaviors = {name: 0 for name in class_names.values()}
    for frame in behavior_history:
        for behavior, count in frame.items():
            total_behaviors[behavior] += count

    # Calculate warning percentage
    warning_percent = (total_warnings / total_frames) * 100 if total_frames > 0 else 0

    # Create plots
    plt.figure(figsize=(15, 8))

    # Behavior frequency plot
    plt.subplot(1, 2, 1)
    behaviors = list(class_names.values())
    counts = [total_behaviors[b] for b in behaviors]
    plt.bar(behaviors, counts, color='red')  # All red for warnings
    plt.xticks(rotation=45, ha='right')
    plt.title('Warning Behavior Frequency')
    plt.ylabel('Count')

    # Warning percentage plot
    plt.subplot(1, 2, 2)
    plt.pie([warning_percent, 100-warning_percent],
            labels=['Warning Frames', 'Normal Frames'],
            colors=['red', 'green'],
            autopct='%1.1f%%')
    plt.title('Warning Frame Percentage')

    plt.tight_layout()
    plt.savefig('behavior_report.png')
    plt.show()

    print("\n=== BEHAVIOR WARNING REPORT ===")
    print(f"Total Frames: {total_frames}")
    print(f"Total Warning Behaviors: {total_warnings} ({warning_percent:.1f}% of frames)")
    print("\nBehavior Details:")
    for behavior, count in total_behaviors.items():
        print(f"WARNING - {behavior}: {count}")

# Process video
video_name = "/content/drive/MyDrive/Colab Notebooks/6601985562796.mp4"
output_video = process_video(video_name)

# Download results
files.download(output_video)
files.download('behavior_report.png')

print("Processing completed! Output video and report downloaded.")


0: 384x640 2 head_downs, 1 looking_right, 12.1ms
Speed: 2.0ms preprocess, 12.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 head_downs, 1 looking_right, 6.6ms
Speed: 2.1ms preprocess, 6.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 head_downs, 1 looking_right, 8.4ms
Speed: 2.1ms preprocess, 8.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 head_downs, 1 looking_right, 8.6ms
Speed: 2.1ms preprocess, 8.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 head_downs, 1 looking_right, 7.1ms
Speed: 2.0ms preprocess, 7.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 head_downs, 1 looking_right, 7.6ms
Speed: 2.0ms preprocess, 7.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 head_downs, 1 looking_right, 8.2ms
Speed: 2.1ms preprocess, 8.2ms inference, 1.4ms postprocess per image at shape 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Hoàn tất xử lý video và tải về.
